In [1]:
import pandas as pd
import numpy as np
import datetime
import glob
import time
import re

In [2]:
start="2009-10-14"

In [3]:
libor=pd.read_csv("LIBOR USD.csv")

In [4]:
libor["Date"]=libor["Date"].map(lambda x:pd.to_datetime(x,format='%d.%m.%Y'))

In [5]:
libor["Month"]=libor["Date"].map(lambda x:x.to_period("M"))

In [6]:
libor=libor.sort_values("Date")

In [7]:
libor=libor[libor["Date"]>start]
libor=libor.groupby("Month")["ON","1W","1M","2M","3M","6M","12M"].mean()

In [8]:
path_trea =r'./Treasury' 
allFiles_trea = glob.glob(path_trea + "/*.csv")
frame_trea = pd.DataFrame()
list_trea = []
for file_trea in allFiles_trea:
    df_trea = pd.read_csv(file_trea,index_col="Date", header=0)
    list_trea.append(df_trea)
frame_trea=pd.concat(list_trea)

In [9]:
frame_trea.index=frame_trea.index.map(lambda x:pd.to_datetime(x,format='%m/%d/%Y'))

In [10]:
frame_trea["Month"]=frame_trea.index.map(lambda x:x.to_period("M"))

In [11]:
frame_trea.to_csv("Treasury_month.csv")

In [12]:
frame_trea=frame_trea[frame_trea.index>start]
frame_trea=frame_trea.groupby("Month")["1 Mo","6 Mo","1 Yr","5 Yr","10 Yr","20 Yr","30 Yr"].mean()

In [13]:
hud=pd.read_csv("RM-A_09-30-2019.csv")

In [14]:
hud["DATE"]=hud["INITIAL ENDORSEMENT DATE"].map(lambda x:pd.to_datetime(x,format='%m/%d/%Y'))

In [15]:
hud["Month"]=hud["DATE"].map(lambda x:x.to_period("M"))

In [16]:
hud=hud[hud["DATE"]>start].reset_index()

In [17]:
cata=["202", "811"]
catb=["542(c)","542(b)"]
catc=["221","220","231","213"]
catd="223(f)"
cate="223(a)(7)"
catg="241a"
cath=["234(d)","207","223(d)"]
cati=["232","Board&Care","ICF","Nursing"]
catj=["Hospital","Group Practice"]

In [18]:
cats=[cata,catb,catc,catd,cate,catg,cath,cati,catj]

In [19]:
for i in range(len(cats)):
    name="Category %i"%(i)
    hud[name]=False
    for j in range(len(hud)):
        for k in range(len(cats[i])):
            if (cats[i][k] in hud["SOA CATEGORY/SUB CATEGORY"][j]):
                hud[name][j]=True

C:\Users\yz402\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


KeyboardInterrupt: 

In [20]:
hud["Category"]=None
for i in range(len(hud)):
    hud["Category"][i]=[]
    if hud["Category 0"][i]==True:
        hud["Category"][i].append("Capital Advances")
    if hud["Category 1"][i]==True:
        hud["Category"][i].append("Risk Sharing")
    if hud["Category 2"][i]==True:
        hud["Category"][i].append("New Construction")
    if hud["Category 3"][i]==True:
        hud["Category"][i].append("Refinancing_Purchase of Apartments_Coops")
    if hud["Category 4"][i]==True:
        hud["Category"][i].append("Refinancing of Apt")
    if hud["Category 5"][i]==True:
        hud["Category"][i].append("Improvements_Additions for Apartments_Coops")
    if hud["Category 6"][i]==True:
        hud["Category"][i].append("Other FHA")
    if hud["Category 7"][i]==True:
        hud["Category"][i].append("Health Care Facilities")
    if hud["Category 8"][i]==True:
        hud["Category"][i].append("Hospitals and Group Practices")

C:\Users\yz402\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
hud.columns

Index(['index', 'HUD PROJECT NUMBER', 'PROPERTY NAME', 'PROPERTY STREET',
       'PROPERTY CITY', 'PROPERTY STATE', 'PROPERTY ZIP', 'UNITS',
       'INITIAL ENDORSEMENT DATE', 'FINAL ENDORSEMENT DATE',
       'ORIGINAL MORTGAGE AMOUNT', 'FIRST PAYMENT DATE', 'MATURITY DATE',
       'TERM IN MONTHS', 'INTEREST RATE', 'CURRENT PRINCIPAL AND INTEREST',
       'AMORTIZED PRINCIPAL BALANCE', 'HOLDER NAME', 'HOLDER CITY',
       'HOLDER STATE', 'SERVICER NAME', 'SERVICER CITY', 'SERVICER STATE',
       'SECTION OF ACT CODE', 'SOA CATEGORY/SUB CATEGORY', 'TE', 'TC', 'DATE',
       'Month', 'Category 0', 'Category 1', 'Category 2', 'Category 3',
       'Category 4', 'Category 5', 'Category 6', 'Category 7', 'Category 8',
       'Category'],
      dtype='object')

In [22]:
hud_cat=hud[['HUD PROJECT NUMBER', 'PROPERTY NAME', 'PROPERTY STREET',
       'PROPERTY CITY', 'PROPERTY STATE', 'PROPERTY ZIP', 'UNITS',
       'INITIAL ENDORSEMENT DATE', 'FINAL ENDORSEMENT DATE',
       'ORIGINAL MORTGAGE AMOUNT', 'FIRST PAYMENT DATE', 'MATURITY DATE',
       'TERM IN MONTHS', 'INTEREST RATE', 'CURRENT PRINCIPAL AND INTEREST',
       'AMORTIZED PRINCIPAL BALANCE', 'HOLDER NAME', 'HOLDER CITY',
       'HOLDER STATE', 'SERVICER NAME', 'SERVICER CITY', 'SERVICER STATE',
       'SECTION OF ACT CODE', 'SOA CATEGORY/SUB CATEGORY','Month','Category']]

In [23]:
hud_cat["PROPERTY CITY"]=hud_cat["PROPERTY CITY"].map(lambda x:x.upper())
hud_cat=hud_cat.sort_values("PROPERTY CITY")


C:\Users\yz402\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
#hud_cat.to_csv("HUD with Category_month.csv")

In [46]:
hud_cat=pd.read_csv("cat_data.csv")

In [47]:
hud_cat["DATE"]=hud_cat["INITIAL ENDORSEMENT DATE"].map(lambda x:pd.to_datetime(x,format='%d/%m/%Y'))
hud_cat["Month"]=hud_cat["DATE"].map(lambda x:x.to_period("M"))

In [118]:
#hud_cat["Address"]=hud_cat["PROPERTY STREET"]+","+hud_cat["PROPERTY STATE"]+","+hud_cat["PROPERTY ZIP"].astype(str)

In [22]:
# hud_cat["Address"]=None

In [23]:
# from geopy.geocoders import Nominatim


In [ ]:

# locas=[]
# err=[]
# for i in range(len(hud_cat)):
#     agent="HUD"+str(i)
#     geolocator = Nominatim(user_agent=agent)
#     time.sleep(1)
#     try:
#         a=hud_cat["PROPERTY STREET"].iloc[i]+","+hud_cat["PROPERTY STATE"].iloc[i]+" "+hud_cat["PROPERTY ZIP"].iloc[i].astype(str)
#         b=r'\(.*?\)'
#         a = re.sub(b, '', a)
#         location = geolocator.geocode(a)
#         locas.append((location.latitude, location.longitude))
#     except:
#         try:
#             a=hud_cat["PROPERTY STREET"].iloc[i]+","+hud_cat["PROPERTY STATE"].iloc[i]+","+hud_cat["PROPERTY CITY"].iloc[i]+" "+hud_cat["PROPERTY ZIP"].iloc[i].astype(str)
#             b=r'\(.*?\)'
#             a = re.sub(b, '', a)
#             location = geolocator.geocode(a)
#             locas.append((location.latitude, location.longitude))
#         except:
#             try:
#                 a=hud_cat["PROPERTY NAME"].iloc[i]+","+hud_cat["PROPERTY STREET"].iloc[i]+","+hud_cat["PROPERTY STATE"].iloc[i]+","+hud_cat["PROPERTY CITY"].iloc[i]+" "+hud_cat["PROPERTY ZIP"].iloc[i].astype(str)
#                 b=r'\(.*?\)'
#                 a = re.sub(b, '', a)
#                 location = geolocator.geocode(a)
#                 locas.append((location.latitude, location.longitude))
#             except:
#                 locas.append((0,0))
#                 err.append(hud_cat.iloc[[i]])  
#                 print(i)



0
2
5
8
13
21
24
30
32
33


,Unnamed: 0,HUD PROJECT NUMBER,PROPERTY NAME,PROPERTY STREET,PROPERTY CITY,PROPERTY STATE,PROPERTY ZIP,UNITS,INITIAL ENDORSEMENT DATE,FINAL ENDORSEMENT DATE,...,HOLDER CITY,HOLDER STATE,SERVICER NAME,SERVICER CITY,SERVICER STATE,SECTION OF ACT CODE,SOA CATEGORY/SUB CATEGORY,Month,Category,Address
8,4036,5235713,NORTH POST COMMONS,ROOSEVELT LINCOLN PRITCHARD AV,ABERDEEN,MD,21001,120,2/27/2013,2/27/2013,...,MILWAUKEE,WI,WELLS FARGO BANK NA,MILWAUKEE,WI,REJ,223(a)(7)/221(d)(4) MKT Refi/ Moderate Income,2013-02,"['New Construction', 'Refinancing_Purchase of ...",None


In [135]:
len(locas)

'RESERVE AT RIVERSIDE PHASE II,PHILADELPHIA ROAD ,MD,ABERDEEN,21001'

In [136]:
loc=geolocator.geocode(aa)
loc.latitude

AttributeError: 'NoneType' object has no attribute 'latitude'

In [123]:
hud_cat["Address"][12487]

'4107 BAYSIDE COURT,IL,60099'

In [53]:
hud_cat

,Unnamed: 0,HUD PROJECT NUMBER,PROPERTY NAME,PROPERTY STREET,PROPERTY CITY,PROPERTY STATE,PROPERTY ZIP,UNITS,INITIAL ENDORSEMENT DATE,FINAL ENDORSEMENT DATE,...,HOLDER CITY,HOLDER STATE,SERVICER NAME,SERVICER CITY,SERVICER STATE,SECTION OF ACT CODE,SOA CATEGORY/SUB CATEGORY,Month,Category,Address
0,7200,7311902,BRIARWICK APARTMENTS,536 SOTHLEA DR.,46902,IN,46902,128,7/26/2017,7/26/2017,...,SAINT LOUIS,MO,GERSHMAN INVESTMENT CORP,SAINT LOUIS,MO,HRI,207/ 223(f) Pur/ Refin Hsg.,2017-07,"['Refinancing_Purchase of Apartments_Coops', '...","BRIARWICK APARTMENTS, 536 SOTHLEA DR., 46902, ..."
1,9094,9235817,ENVI,3200 SOUTHDALE CIRCLE,55435,MN,55435,184,12/15/2017,NaN,...,SAINT PAUL,MN,JONES LANG LASALLE,SAINT PAUL,MN,OMI,221(d)(4) Mkt. Rate Mod Inc/ Disp Fams,2017-12,"['New Construction', 'Refinancing_Purchase of ...","ENVI, 3200 SOUTHDALE CIRCLE, 55435, MN, 55435"
2,5593,6435572,LIVE OAK MANOR,1610 E. MARTIN LUTHER KING JR.,ABBEVILLE,LA,70510,119,3/12/2013,6/23/2014,...,BIRMINGHAM,AL,HIGHLAND COMMERCIAL MORTGAGE L,BIRMINGHAM,AL,OMI,221(d)(4) Mkt. Rate Mod Inc/ Disp Fams,2013-03,"['New Construction', 'Refinancing_Purchase of ...","LIVE OAK MANOR, 1610 E. MARTIN LUTHER KING JR...."
3,12052,12722100,GRACYS HARBOR HEALTH & REHAB C,930 ANDERSON DRIVE,ABERCEN,WA,98520,105,8/29/2013,8/29/2013,...,BALTIMORE,MD,CAPITAL FUNDING LLC,BALTIMORE,MD,RFP,223(a)(7)/232/223(f)/Pur/Refi/Nursing Home,2013-08,"['Refinancing_Purchase of Apartments_Coops', '...","GRACYS HARBOR HEALTH & REHAB C, 930 ANDERSON D..."
4,4153,5298034,FAIRBROOKE APARTMENTS,700 WEST BEL AIR AVENUE,ABERDEEN,MD,21001,122,12/17/2014,12/17/2014,...,CROWNSVILLE,MD,DEPARTMENT HOUSING COMM DEV,CROWNSVILLE,MD,YHN,542(c) HFA Risk Sharing-Recent Comp,2014-12,"['Risk Sharing', 'Refinancing_Purchase of Apar...","FAIRBROOKE APARTMENTS, 700 WEST BEL AIR AVENUE..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12487,6232,7111237,HORIZON VILLAGE APTS,4107 BAYSIDE COURT,ZION,IL,60099,276,8/19/2011,8/19/2011,...,MIAMISBURG,OH,"PNC BANK, NATIONAL ASSOCIATION",MIAMISBURG,OH,REK,223(a)(7)/207/223(f) Refinanced Insurance,2011-08,"['Refinancing_Purchase of Apartments_Coops', '...","HORIZON VILLAGE APTS, 4107 BAYSIDE COURT, ZION..."
12488,6275,7111310,ZION SENIOR COTTAGE,1005 PRESTWICK STREET,ZION,IL,60099,110,5/31/2013,5/31/2013,...,MIAMISBURG,OH,"PNC BANK, NATIONAL ASSOCIATION",MIAMISBURG,OH,HRI,207/ 223(f) Pur/ Refin Hsg.,2013-05,"['Refinancing_Purchase of Apartments_Coops', '...","ZION SENIOR COTTAGE, 1005 PRESTWICK STREET, ZI..."
12489,7124,7311812,WESTHAVEN PHASE II,7105 WEST HAVEN PHASE II,ZIONSVILLE,IN,46077,66,2/27/2015,2/27/2015,...,CARMEL,IN,MERCHANTS CAPITAL CORP.,CARMEL,IN,HRI,207/ 223(f) Pur/ Refin Hsg.,2015-02,"['Refinancing_Purchase of Apartments_Coops', '...","WESTHAVEN PHASE II, 7105 WEST HAVEN PHASE II, ..."
12490,7075,7311753,THE RESERVE AT WILLIAMS GLEN,2201 WILLIAMS GLEN BLVD,ZIONSVILLE,IN,46077,268,5/23/2014,5/23/2014,...,CARMEL,IN,MERCHANTS CAPITAL CORP.,CARMEL,IN,HRI,207/ 223(f) Pur/ Refin Hsg.,2014-05,"['Refinancing_Purchase of Apartments_Coops', '...","THE RESERVE AT WILLIAMS GLEN, 2201 WILLIAMS G..."


In [26]:
# capital advance
CA=hud[hud["Category 0"]==True].groupby(["PROPERTY CITY","Month"])["INTEREST RATE"].mean()

In [27]:
aa=hud.groupby(["PROPERTY CITY","Month"])["INTEREST RATE"].mean()
aa=aa.reset_index(["PROPERTY CITY","Month"])

In [28]:
aa.to_csv("City Data for Risk Sharing with HFAs & QPEs Type Loan_month.csv")

In [29]:
# Risk Sharing with HFAs & QPEs
RS=hud[hud["Category 1"]==True].groupby(["PROPERTY STATE","Month"])["INTEREST RATE"].mean()
RS=RS.reset_index(["PROPERTY STATE","Month"])

In [30]:
states=RS["PROPERTY STATE"].unique()
df_list=[]
df_s=pd.DataFrame()
for i in range(len(states)):
    df=RS[RS["PROPERTY STATE"]==states[i]]
    df=df.merge(libor,on="Month")
    df["spread between libor 12M"]=df["INTEREST RATE"]-df["12M"]
    df["Corr with libor 12M"]=df["12M"].corr(df["INTEREST RATE"])
    df=df.merge(frame_trea,on="Month")
    df["spread between 10 Yr treasury"]=df["INTEREST RATE"]-df["10 Yr"]
    df["Corr with 10 Yr treasury"]=df["10 Yr"].corr(df["INTEREST RATE"])
    df["spread between 20 Yr treasury"]=df["INTEREST RATE"]-df["20 Yr"]
    df["Corr with 20 Yr treasury"]=df["20 Yr"].corr(df["INTEREST RATE"])
    df["spread between 30 Yr treasury"]=df["INTEREST RATE"]-df["30 Yr"]
    df["Corr with 30 Yr treasury"]=df["30 Yr"].corr(df["INTEREST RATE"])
    df_list.append(df)
df_s=pd.concat(df_list).reset_index(drop=True)
    

C:\Users\yz402\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\yz402\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [31]:
df_s.to_csv("State Data for Risk Sharing with HFAs & QPEs Type Loan_month.csv")

In [48]:
hud_cat["coor"]=list(zip(hud_cat.longitude, hud_cat.latitude))

In [44]:
# aa=hud_cat.groupby(["PROPERTY CITY","Month"]).agg({'INTEREST RATE':['mean'],'coor':['first']})
# aa=aa.reset_index(["PROPERTY CITY","Month"])

In [45]:
aa

,PROPERTY CITY,Month,INTEREST RATE,coor
,,,mean,first
0,13794 21 MILE ROAD,2018-08,420.00,"(28.046670000000002, 53.528330000000004)"
1,438,2019-09,420.00,"(-74.009902, 40.651365000000006)"
2,46902,2017-07,3.28,"(-86.137978, 40.44325)"
3,55435,2017-12,3.66,"(-93.32030300000001, 44.883077)"
4,ABBEVILLE,2013-03,3.15,"(-92.122399, 29.963283)"
...,...,...,...,...
11337,elko,2019-02,4.25,"(-115.787848, 40.848025)"
11338,kokomo,2017-05,3.57,"(-86.161467, 40.494478)"
11339,martinsburg,2017-05,3.75,"(-77.9876, 39.457297)"


In [38]:
hud=hud.groupby(["PROPERTY CITY","Month"])["INTEREST RATE"].mean()
hud=hud.reset_index(["PROPERTY CITY","Month"])
hud

,PROPERTY CITY,Month,INTEREST RATE
0,46902,2017-07,3.28
1,55435,2017-12,3.66
2,ABBEVILLE,2013-03,3.15
3,ABERCEN,2013-08,4.89
4,ABERDEEN,2013-02,3.21
...,...,...,...
11337,elko,2019-02,4.25
11338,kokomo,2017-05,3.57
11339,martinsburg,2017-05,3.75
11340,michigan city,2017-07,3.25


In [19]:
at_vol=pd.read_csv("auto/vol_data_AUT.csv")
at_vol["date"]=at_vol["date"].map(lambda x:pd.to_datetime(x,format='%Y-%m'))
at_vol["Month"]=at_vol["date"].map(lambda x:x.to_period("M"))
at_vol=at_vol.drop(columns={"month",'date'})
at_vol=at_vol.rename(columns={"vol":"auto_vol"})
at_vol["auto_vol(-1)"]=at_vol["auto_vol"].shift(1)
at_vol["auto_vol(-2)"]=at_vol["auto_vol"].shift(2)
at_vol["auto_vol(-3)"]=at_vol["auto_vol"].shift(3)
at_vol["auto_vol(-4)"]=at_vol["auto_vol"].shift(4)
at_vol["auto_vol(1)"]=at_vol["auto_vol"].shift(-1)
at_vol["auto_vol(2)"]=at_vol["auto_vol"].shift(-2)
at_vol["auto_vol(3)"]=at_vol["auto_vol"].shift(-3)
at_vol["auto_vol(4)"]=at_vol["auto_vol"].shift(-4)

In [20]:
cred=pd.read_csv("credit card/tightness.csv")
cred["date"]=cred["date"].map(lambda x:pd.to_datetime(x,format="%Y-%m"))
cred["Month"]=cred["date"].map(lambda x: x.to_period("M"))
cred=cred.drop(columns={"date","month"})
cred=cred.rename(columns={"tightness_index":"credit_tightness"})
cred["credit_tightness(-1)"]=cred["credit_tightness"].shift(1)
cred["credit_tightness(-2)"]=cred["credit_tightness"].shift(2)
cred["credit_tightness(-3)"]=cred["credit_tightness"].shift(3)
cred["credit_tightness(-4)"]=cred["credit_tightness"].shift(4)
cred["credit_tightness(1)"]=cred["credit_tightness"].shift(-1)
cred["credit_tightness(2)"]=cred["credit_tightness"].shift(-2)
cred["credit_tightness(3)"]=cred["credit_tightness"].shift(-3)
cred["credit_tightness(4)"]=cred["credit_tightness"].shift(-4)

In [22]:
stu=pd.read_csv("student/vol_data_STU.csv")
stu["date"]=stu["date"].map(lambda x:pd.to_datetime(x,format="%Y-%m"))
stu["Month"]=stu["date"].map(lambda x:x.to_period("M"))
stu=stu.drop(columns={"month",'date'})
stu=stu.rename(columns={"vol":"student_vol"})
stu["student_vol(-1)"]=stu["student_vol"].shift(1)
stu["student_vol(-2)"]=stu["student_vol"].shift(2)
stu["student_vol(-3)"]=stu["student_vol"].shift(3)
stu["student_vol(-4)"]=stu["student_vol"].shift(4)
stu["student_vol(1)"]=stu["student_vol"].shift(-1)
stu["student_vol(2)"]=stu["student_vol"].shift(-2)
stu["student_vol(3)"]=stu["student_vol"].shift(-3)
stu["student_vol(4)"]=stu["student_vol"].shift(-4)

In [50]:
city=hud["PROPERTY CITY"].unique()
df_list=[]
df_s=pd.DataFrame()
for i in range(len(city)):
    df=hud[hud["PROPERTY CITY"]==city[i]]
    df=df.merge(libor["ON"],on="Month")
    df["spread between libor ON"]=df["INTEREST RATE"]-df["ON"]
    df["Corr with libor ON"]=df["ON"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["1W"],on="Month")
    df["spread between libor 1W"]=df["INTEREST RATE"]-df["1W"]
    df["Corr with libor 1W"]=df["1W"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["1M"],on="Month")
    df["spread between libor 1M"]=df["INTEREST RATE"]-df["1M"]
    df["Corr with libor 1M"]=df["1M"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["2M"],on="Month")
    df["spread between libor 2M"]=df["INTEREST RATE"]-df["2M"]
    df["Corr with libor 2M"]=df["2M"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["3M"],on="Month")
    df["spread between libor 3M"]=df["INTEREST RATE"]-df["3M"]
    df["Corr with libor 3M"]=df["3M"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["6M"],on="Month")
    df["spread between libor 6M"]=df["INTEREST RATE"]-df["6M"]
    df["Corr with libor 6M"]=df["6M"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["12M"],on="Month")
    df["spread between libor 12M"]=df["INTEREST RATE"]-df["12M"]
    df["Corr with libor 12M"]=df["12M"].corr(df["INTEREST RATE"])

    df=df.merge(frame_trea["1 Mo"],on="Month")
    df["spread between 1 Mo treasury"]=df["INTEREST RATE"]-df["1 Mo"]
    df["Corr with 1 Mo treasury"]=df["1 Mo"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["6 Mo"],on="Month")
    df["spread between 6 Mo treasury"]=df["INTEREST RATE"]-df["6 Mo"]
    df["Corr with 6 Mo treasury"]=df["6 Mo"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["1 Yr"],on="Month")
    df["spread between 1 Yr treasury"]=df["INTEREST RATE"]-df["1 Yr"]
    df["Corr with 1 Yr treasury"]=df["1 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["5 Yr"],on="Month")
    df["spread between 5 Yr treasury"]=df["INTEREST RATE"]-df["5 Yr"]
    df["Corr with 5 Yr treasury"]=df["5 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["10 Yr"],on="Month")
    df["spread between 10 Yr treasury"]=df["INTEREST RATE"]-df["10 Yr"]
    df["Corr with 10 Yr treasury"]=df["10 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["20 Yr"],on="Month")
    df["spread between 20 Yr treasury"]=df["INTEREST RATE"]-df["20 Yr"]
    df["Corr with 20 Yr treasury"]=df["20 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["30 Yr"],on="Month")
    df["spread between 30 Yr treasury"]=df["INTEREST RATE"]-df["30 Yr"]
    df["Corr with 30 Yr treasury"]=df["30 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol","Month"]],on="Month")
    df["Corr with auto loan volume"]=df["auto_vol"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol(-1)","Month"]],on="Month")
    df["Corr with lag 1 auto loan volume"]=df["auto_vol(-1)"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol(-2)","Month"]],on="Month")
    df["Corr with lag 2 auto loan volume"]=df["auto_vol(-2)"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol(-3)","Month"]],on="Month")
    df["Corr with lag 3 auto loan volume"]=df["auto_vol(-3)"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol(-4)","Month"]],on="Month")
    df["Corr with lag 4 auto loan volume"]=df["auto_vol(-4)"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol(1)","Month"]],on="Month")
    df["Corr with lead 1 auto loan volume"]=df["auto_vol(1)"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol(2)","Month"]],on="Month")
    df["Corr with lead 2 auto loan volume"]=df["auto_vol(2)"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol(3)","Month"]],on="Month")
    df["Corr with lead 3 auto loan volume"]=df["auto_vol(3)"].corr(df["INTEREST RATE"])
    
    df=df.merge(at_vol[["auto_vol(4)","Month"]],on="Month")
    df["Corr with lead 4 auto loan volume"]=df["auto_vol(4)"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol","Month"]],on="Month")
    df["Corr with student loan volume"]=df["student_vol"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol(-1)","Month"]],on="Month")
    df["Corr with lag 1 student loan volume"]=df["student_vol(-1)"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol(-2)","Month"]],on="Month")
    df["Corr with lag 2 student loan volume"]=df["student_vol(-2)"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol(-3)","Month"]],on="Month")
    df["Corr with lag 3 student loan volume"]=df["student_vol(-3)"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol(-4)","Month"]],on="Month")
    df["Corr with lag 4 student loan volume"]=df["student_vol(-4)"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol(1)","Month"]],on="Month")
    df["Corr with lead 1 student loan volume"]=df["student_vol(1)"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol(2)","Month"]],on="Month")
    df["Corr with lead 2 student loan volume"]=df["student_vol(2)"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol(3)","Month"]],on="Month")
    df["Corr with lead 3 student loan volume"]=df["student_vol(3)"].corr(df["INTEREST RATE"])
    
    df=df.merge(stu[["student_vol(4)","Month"]],on="Month")
    df["Corr with lead 4 student loan volume"]=df["student_vol(4)"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness","Month"]],on="Month")
    df["Corr with credit tightness"]=df["credit_tightness"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness(-1)","Month"]],on="Month")
    df["Corr with lag 1 credit tightness"]=df["credit_tightness(-1)"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness(-2)","Month"]],on="Month")
    df["Corr with lag 2 credit tightness"]=df["credit_tightness(-2)"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness(-3)","Month"]],on="Month")
    df["Corr with lag 3 credit tightness"]=df["credit_tightness(-3)"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness(-4)","Month"]],on="Month")
    df["Corr with lag 4 credit tightness"]=df["credit_tightness(-4)"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness(1)","Month"]],on="Month")
    df["Corr with lead 1 credit tightness"]=df["credit_tightness(1)"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness(2)","Month"]],on="Month")
    df["Corr with lead 2 credit tightness"]=df["credit_tightness(2)"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness(3)","Month"]],on="Month")
    df["Corr with lead 3 credit tightness"]=df["credit_tightness(3)"].corr(df["INTEREST RATE"])
    
    df=df.merge(cred[["credit_tightness(4)","Month"]],on="Month")
    df["Corr with lead 4 credit tightness"]=df["credit_tightness(4)"].corr(df["INTEREST RATE"])
    df_list.append(df)
df_s=pd.concat(df_list).reset_index(drop=True)
df_s=df_s[['PROPERTY CITY', 'Month', 'INTEREST RATE', 'ON',
       'spread between libor ON', 'Corr with libor ON', '1W',
       'spread between libor 1W', 'Corr with libor 1W', '1M',
       'spread between libor 1M', 'Corr with libor 1M', '2M',
       'spread between libor 2M', 'Corr with libor 2M', '3M',
       'spread between libor 3M', 'Corr with libor 3M', '6M',
       'spread between libor 6M', 'Corr with libor 6M', '12M',
       'spread between libor 12M', 'Corr with libor 12M', '1 Mo',
       'spread between 1 Mo treasury', 'Corr with 1 Mo treasury', '6 Mo',
       'spread between 6 Mo treasury', 'Corr with 6 Mo treasury', '1 Yr',
       'spread between 1 Yr treasury', 'Corr with 1 Yr treasury', '5 Yr',
       'spread between 5 Yr treasury', 'Corr with 5 Yr treasury', '10 Yr',
       'spread between 10 Yr treasury', 'Corr with 10 Yr treasury', '20 Yr',
       'spread between 20 Yr treasury', 'Corr with 20 Yr treasury', '30 Yr',
       'spread between 30 Yr treasury', 'Corr with 30 Yr treasury', 'auto_vol',
       'Corr with auto loan volume', 'auto_vol(-1)',
       'Corr with lag 1 auto loan volume', 'auto_vol(-2)',
       'Corr with lag 2 auto loan volume', 'auto_vol(-3)',
       'Corr with lag 3 auto loan volume', 'auto_vol(-4)',
       'Corr with lag 4 auto loan volume', 'auto_vol(1)',
       'Corr with lead 1 auto loan volume', 'auto_vol(2)',
       'Corr with lead 2 auto loan volume', 'auto_vol(3)',
       'Corr with lead 3 auto loan volume', 'auto_vol(4)',
       'Corr with lead 4 auto loan volume', 'student_vol',
       'Corr with student loan volume', 'student_vol(-1)',
       'Corr with lag 1 student loan volume', 'student_vol(-2)',
       'Corr with lag 2 student loan volume', 'student_vol(-3)',
       'Corr with lag 3 student loan volume', 'student_vol(-4)',
       'Corr with lag 4 student loan volume', 'student_vol(1)',
       'Corr with lead 1 student loan volume', 'student_vol(2)',
       'Corr with lead 2 student loan volume', 'student_vol(3)',
       'Corr with lead 3 student loan volume', 'student_vol(4)',
       'Corr with lead 4 student loan volume', 'credit_tightness',
       'Corr with credit tightness', 'credit_tightness(-1)',
       'Corr with lag 1 credit tightness', 'credit_tightness(-2)',
       'Corr with lag 2 credit tightness', 'credit_tightness(-3)',
       'Corr with lag 3 credit tightness', 'credit_tightness(-4)',
       'Corr with lag 4 credit tightness', 'credit_tightness(1)',
       'Corr with lead 1 credit tightness', 'credit_tightness(2)',
       'Corr with lead 2 credit tightness', 'credit_tightness(3)',
       'Corr with lead 3 credit tightness', 'credit_tightness(4)',
       'Corr with lead 4 credit tightness']]

df_s.to_csv("month/City Data of HUD_month.csv",index=False)

C:\Users\yz402\Anaconda3\lib\site-packages\ipykernel_launcher.py:143: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [51]:
categories=["Risk Sharing","New Construction","Refinancing_Purchase of Apartments_Coops",
     "Refinancing of Apt","Improvements_Additions for Apartments_Coops","Other FHA","Health Care Facilities",
     "Hospitals and Group Practices"]

for i in range(len(categories)):
    hud_sub=hud_cat[hud_cat["Category"].str.contains(categories[i],regex=False)]
    hud_sub=hud_sub.groupby(['PROPERTY STATE',"PROPERTY CITY","Month"]).agg({'INTEREST RATE':['mean'],'coor':['first']})
    hud_sub=hud_sub.reset_index(['PROPERTY STATE',"PROPERTY CITY","Month"])
    hud_sub["Category"]=categories[i]
    city=hud_sub["PROPERTY CITY"].unique()
    df_list=[]
    df_s=pd.DataFrame()
    name="month/City Data of HUD_"+categories[i]
    for j in range(len(city)):
        df=hud_sub[hud_sub["PROPERTY CITY"]==city[j]]
        df=df.merge(libor["ON"],on="Month")
        df["spread between libor ON"]=df["INTEREST RATE"]-df["ON"]
        df["Corr with libor ON"]=df["ON"].corr(df["INTEREST RATE"])

        df=df.merge(libor["1W"],on="Month")
        df["spread between libor 1W"]=df["INTEREST RATE"]-df["1W"]
        df["Corr with libor 1W"]=df["1W"].corr(df["INTEREST RATE"])

        df=df.merge(libor["1M"],on="Month")
        df["spread between libor 1M"]=df["INTEREST RATE"]-df["1M"]
        df["Corr with libor 1M"]=df["1M"].corr(df["INTEREST RATE"])

        df=df.merge(libor["2M"],on="Month")
        df["spread between libor 2M"]=df["INTEREST RATE"]-df["2M"]
        df["Corr with libor 2M"]=df["2M"].corr(df["INTEREST RATE"])

        df=df.merge(libor["3M"],on="Month")
        df["spread between libor 3M"]=df["INTEREST RATE"]-df["3M"]
        df["Corr with libor 3M"]=df["3M"].corr(df["INTEREST RATE"])

        df=df.merge(libor["6M"],on="Month")
        df["spread between libor 6M"]=df["INTEREST RATE"]-df["6M"]
        df["Corr with libor 6M"]=df["6M"].corr(df["INTEREST RATE"])

        df=df.merge(libor["12M"],on="Month")
        df["spread between libor 12M"]=df["INTEREST RATE"]-df["12M"]
        df["Corr with libor 12M"]=df["12M"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["1 Mo"],on="Month")
        df["spread between 1 Mo treasury"]=df["INTEREST RATE"]-df["1 Mo"]
        df["Corr with 1 Mo treasury"]=df["1 Mo"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["6 Mo"],on="Month")
        df["spread between 6 Mo treasury"]=df["INTEREST RATE"]-df["6 Mo"]
        df["Corr with 6 Mo treasury"]=df["6 Mo"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["1 Yr"],on="Month")
        df["spread between 1 Yr treasury"]=df["INTEREST RATE"]-df["1 Yr"]
        df["Corr with 1 Yr treasury"]=df["1 Yr"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["5 Yr"],on="Month")
        df["spread between 5 Yr treasury"]=df["INTEREST RATE"]-df["5 Yr"]
        df["Corr with 5 Yr treasury"]=df["5 Yr"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["10 Yr"],on="Month")
        df["spread between 10 Yr treasury"]=df["INTEREST RATE"]-df["10 Yr"]
        df["Corr with 10 Yr treasury"]=df["10 Yr"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["20 Yr"],on="Month")
        df["spread between 20 Yr treasury"]=df["INTEREST RATE"]-df["20 Yr"]
        df["Corr with 20 Yr treasury"]=df["20 Yr"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["30 Yr"],on="Month")
        df["spread between 30 Yr treasury"]=df["INTEREST RATE"]-df["30 Yr"]
        df["Corr with 30 Yr treasury"]=df["30 Yr"].corr(df["INTEREST RATE"])
        
        df=df.merge(at_vol[["auto_vol","Month"]],on="Month")
        df["Corr with auto loan volume"]=df["auto_vol"].corr(df["INTEREST RATE"])

        df=df.merge(at_vol[["auto_vol(-1)","Month"]],on="Month")
        df["Corr with lag 1 auto loan volume"]=df["auto_vol(-1)"].corr(df["INTEREST RATE"])

        df=df.merge(at_vol[["auto_vol(-2)","Month"]],on="Month")
        df["Corr with lag 2 auto loan volume"]=df["auto_vol(-2)"].corr(df["INTEREST RATE"])

        df=df.merge(at_vol[["auto_vol(-3)","Month"]],on="Month")
        df["Corr with lag 3 auto loan volume"]=df["auto_vol(-3)"].corr(df["INTEREST RATE"])

        df=df.merge(at_vol[["auto_vol(-4)","Month"]],on="Month")
        df["Corr with lag 4 auto loan volume"]=df["auto_vol(-4)"].corr(df["INTEREST RATE"])

        df=df.merge(at_vol[["auto_vol(1)","Month"]],on="Month")
        df["Corr with lead 1 auto loan volume"]=df["auto_vol(1)"].corr(df["INTEREST RATE"])

        df=df.merge(at_vol[["auto_vol(2)","Month"]],on="Month")
        df["Corr with lead 2 auto loan volume"]=df["auto_vol(2)"].corr(df["INTEREST RATE"])

        df=df.merge(at_vol[["auto_vol(3)","Month"]],on="Month")
        df["Corr with lead 3 auto loan volume"]=df["auto_vol(3)"].corr(df["INTEREST RATE"])

        df=df.merge(at_vol[["auto_vol(4)","Month"]],on="Month")
        df["Corr with lead 4 auto loan volume"]=df["auto_vol(4)"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol","Month"]],on="Month")
        df["Corr with student loan volume"]=df["student_vol"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol(-1)","Month"]],on="Month")
        df["Corr with lag 1 student loan volume"]=df["student_vol(-1)"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol(-2)","Month"]],on="Month")
        df["Corr with lag 2 student loan volume"]=df["student_vol(-2)"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol(-3)","Month"]],on="Month")
        df["Corr with lag 3 student loan volume"]=df["student_vol(-3)"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol(-4)","Month"]],on="Month")
        df["Corr with lag 4 student loan volume"]=df["student_vol(-4)"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol(1)","Month"]],on="Month")
        df["Corr with lead 1 student loan volume"]=df["student_vol(1)"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol(2)","Month"]],on="Month")
        df["Corr with lead 2 student loan volume"]=df["student_vol(2)"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol(3)","Month"]],on="Month")
        df["Corr with lead 3 student loan volume"]=df["student_vol(3)"].corr(df["INTEREST RATE"])

        df=df.merge(stu[["student_vol(4)","Month"]],on="Month")
        df["Corr with lead 4 student loan volume"]=df["student_vol(4)"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness","Month"]],on="Month")
        df["Corr with credit tightness"]=df["credit_tightness"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness(-1)","Month"]],on="Month")
        df["Corr with lag 1 credit tightness"]=df["credit_tightness(-1)"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness(-2)","Month"]],on="Month")
        df["Corr with lag 2 credit tightness"]=df["credit_tightness(-2)"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness(-3)","Month"]],on="Month")
        df["Corr with lag 3 credit tightness"]=df["credit_tightness(-3)"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness(-4)","Month"]],on="Month")
        df["Corr with lag 4 credit tightness"]=df["credit_tightness(-4)"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness(1)","Month"]],on="Month")
        df["Corr with lead 1 credit tightness"]=df["credit_tightness(1)"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness(2)","Month"]],on="Month")
        df["Corr with lead 2 credit tightness"]=df["credit_tightness(2)"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness(3)","Month"]],on="Month")
        df["Corr with lead 3 credit tightness"]=df["credit_tightness(3)"].corr(df["INTEREST RATE"])

        df=df.merge(cred[["credit_tightness(4)","Month"]],on="Month")
        df["Corr with lead 4 credit tightness"]=df["credit_tightness(4)"].corr(df["INTEREST RATE"])
        df_list.append(df)
    if len(df_list)!=0:
        df_s=pd.concat(df_list).reset_index(drop=True)
        df_s=df_s[['PROPERTY CITY', 'Month', 'INTEREST RATE', 'Category', 'ON',
       'spread between libor ON', 'Corr with libor ON', '1W',
       'spread between libor 1W', 'Corr with libor 1W', '1M',
       'spread between libor 1M', 'Corr with libor 1M', '2M',
       'spread between libor 2M', 'Corr with libor 2M', '3M',
       'spread between libor 3M', 'Corr with libor 3M', '6M',
       'spread between libor 6M', 'Corr with libor 6M', '12M',
       'spread between libor 12M', 'Corr with libor 12M', '1 Mo',
       'spread between 1 Mo treasury', 'Corr with 1 Mo treasury', '6 Mo',
       'spread between 6 Mo treasury', 'Corr with 6 Mo treasury', '1 Yr',
       'spread between 1 Yr treasury', 'Corr with 1 Yr treasury', '5 Yr',
       'spread between 5 Yr treasury', 'Corr with 5 Yr treasury', '10 Yr',
       'spread between 10 Yr treasury', 'Corr with 10 Yr treasury', '20 Yr',
       'spread between 20 Yr treasury', 'Corr with 20 Yr treasury', '30 Yr',
       'spread between 30 Yr treasury', 'Corr with 30 Yr treasury', 'auto_vol',
       'Corr with auto loan volume', 'auto_vol(-1)',
       'Corr with lag 1 auto loan volume', 'auto_vol(-2)',
       'Corr with lag 2 auto loan volume', 'auto_vol(-3)',
       'Corr with lag 3 auto loan volume', 'auto_vol(-4)',
       'Corr with lag 4 auto loan volume', 'auto_vol(1)',
       'Corr with lead 1 auto loan volume', 'auto_vol(2)',
       'Corr with lead 2 auto loan volume', 'auto_vol(3)',
       'Corr with lead 3 auto loan volume', 'auto_vol(4)',
       'Corr with lead 4 auto loan volume', 'student_vol',
       'Corr with student loan volume', 'student_vol(-1)',
       'Corr with lag 1 student loan volume', 'student_vol(-2)',
       'Corr with lag 2 student loan volume', 'student_vol(-3)',
       'Corr with lag 3 student loan volume', 'student_vol(-4)',
       'Corr with lag 4 student loan volume', 'student_vol(1)',
       'Corr with lead 1 student loan volume', 'student_vol(2)',
       'Corr with lead 2 student loan volume', 'student_vol(3)',
       'Corr with lead 3 student loan volume', 'student_vol(4)',
       'Corr with lead 4 student loan volume', 'credit_tightness',
       'Corr with credit tightness', 'credit_tightness(-1)',
       'Corr with lag 1 credit tightness', 'credit_tightness(-2)',
       'Corr with lag 2 credit tightness', 'credit_tightness(-3)',
       'Corr with lag 3 credit tightness', 'credit_tightness(-4)',
       'Corr with lag 4 credit tightness', 'credit_tightness(1)',
       'Corr with lead 1 credit tightness', 'credit_tightness(2)',
       'Corr with lead 2 credit tightness', 'credit_tightness(3)',
       'Corr with lead 3 credit tightness', 'credit_tightness(4)',
       'Corr with lead 4 credit tightness']]
        df_s.to_csv(name+"_month.csv",index=False)

C:\Users\yz402\Anaconda3\lib\site-packages\ipykernel_launcher.py:154: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [ ]:
cities=["BOSTON", "PHILADELPHIA", "NEW YORK", "WASHINGTON", "CHICAGO", "LOS ANGELES", "ATLANTA"]